In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import xgboost
import pandas as pd
from glob import glob

!pip install --pre pycaret

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Sports Betting Research/Data
!ls

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 487 kB 10.2 MB/s 
     |████████████████████████████████| 2.0 MB 28.3 MB/s 
     |████████████████████████████████| 1.5 MB 13.3 MB/s 
     |████████████████████████████████| 3.4 MB 49.8 MB/s 
     |████████████████████████████████| 141 kB 76.0 MB/s 
     |████████████████████████████████| 79.9 MB 140 kB/s 
     |████████████████████████████████| 45 kB 1.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 106 kB 88.1 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 72 kB 624 kB/s 
     |████████████████████████████████| 280 kB 61.7 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 9.4 MB 58.5 MB/s 
     |███████████████████████

Mounted at /content/drive
/content/drive/MyDrive/Sports Betting Research/Data
2012_2013_add_dummy_cols.csv   2012_2013_drop_features.csv
2012_2013_add_noise.csv        2012_2013_duplicate_rows.csv
2012_2013_bootstrapped.csv     2012-2013_RawScores_VegasOdds.csv
2012_2013_Cumulative_Data.csv  logs.log


In [2]:
def merge_data(data, column):

  df = pd.DataFrame()

  for gameid in list(data[column].unique()):
    data1 = data.loc[data[column] == gameid][:1]
    data2 = data.loc[data[column] == gameid][1:2]

    merged_df = pd.merge(data1,data2, on=column)
    df = pd.concat([df, merged_df], ignore_index=True)

  # Now fix the odds column

  df['ABS_Line_ML'] = abs(df['Average_Line_ML_x']) + abs(df['Average_Line_ML_y'])

  #Take the difference of the columns
    
  return df

In [3]:
path = '2012_2013_Cumulative_Data.csv'

data = pd.read_csv(path)

data = data.dropna()

data




,Unnamed: 0,Date,TeamId,GameId,PTS_QTR1,PTS_QTR2,PTS_QTR3,PTS_QTR4,PTS,FG_PCT,...,PercentBet_ML,Open_Line_ML,Pinnacle_ML,5dimes_ML,Heritage_ML,Bovada_ML,Betonline_ML,Average_Line_ML,Best_Line_ML,Worst_Line_ML
0,0,2012-10-30,1610612764,21200001,24.000000,15.000000,23.000000,22.000000,84.000000,0.356000,...,44.6,220.0,210.0,200,210,215.0,210.0,209.0,215.0,200.0
1,1,2012-10-30,1610612739,21200001,31.000000,19.000000,24.000000,20.000000,94.000000,0.456000,...,55.4,-260.0,-235.0,-240,-250,-255.0,-250.0,-246.0,-235.0,-255.0
2,2,2012-10-30,1610612738,21200002,25.000000,29.000000,22.000000,31.000000,107.000000,0.520000,...,44.8,220.0,244.0,240,230,225.0,235.0,234.8,244.0,225.0
3,3,2012-10-30,1610612748,21200002,31.000000,31.000000,31.000000,27.000000,120.000000,0.544000,...,55.2,-260.0,-275.0,-260,-270,-265.0,-280.0,-270.0,-260.0,-280.0
4,4,2012-10-30,1610612742,21200003,25.000000,23.000000,26.000000,25.000000,99.000000,0.471000,...,40.8,350.0,393.0,420,400,375.0,385.0,394.6,420.0,375.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2449,2449,2013-04-17,1610612760,21201226,26.719512,27.109756,25.878049,25.121951,105.719512,0.481415,...,100.0,-190.0,-153.0,-150,-160,-155.0,-160.0,-155.6,-150.0,-160.0
2454,2454,2013-04-17,1610612746,21201229,25.512195,26.012195,24.829268,24.560976,101.085366,0.478000,...,33.3,-260.0,-250.0,-290,-305,-280.0,-300.0,-285.0,-250.0,-305.0
2455,2455,2013-04-17,1610612758,21201229,24.951220,24.317073,24.951220,25.500000,100.231707,0.448049,...,66.7,220.0,222.0,260,255,240.0,250.0,245.4,260.0,222.0
2456,2456,2013-04-17,1610612744,21201230,25.317073,25.975610,24.756098,24.500000,101.170732,0.458549,...,40.4,-380.0,-380.0,-400,-405,-420.0,-400.0,-401.0,-380.0,-420.0


In [6]:
#single_game = merge_data(data,'GameId')
single_game.columns

Index(['Unnamed: 0_x', 'Date_x', 'TeamId_x', 'GameId', 'PTS_QTR1_x',
       'PTS_QTR2_x', 'PTS_QTR3_x', 'PTS_QTR4_x', 'PTS_x', 'FG_PCT_x',
       'FT_PCT_x', 'FG3_PCT_x', 'AST_x', 'REB_x', 'TOV_x', 'PercentBet_ML_x',
       'Open_Line_ML_x', 'Pinnacle_ML_x', '5dimes_ML_x', 'Heritage_ML_x',
       'Bovada_ML_x', 'Betonline_ML_x', 'Average_Line_ML_x', 'Best_Line_ML_x',
       'Worst_Line_ML_x', 'Unnamed: 0_y', 'Date_y', 'TeamId_y', 'PTS_QTR1_y',
       'PTS_QTR2_y', 'PTS_QTR3_y', 'PTS_QTR4_y', 'PTS_y', 'FG_PCT_y',
       'FT_PCT_y', 'FG3_PCT_y', 'AST_y', 'REB_y', 'TOV_y', 'PercentBet_ML_y',
       'Open_Line_ML_y', 'Pinnacle_ML_y', '5dimes_ML_y', 'Heritage_ML_y',
       'Bovada_ML_y', 'Betonline_ML_y', 'Average_Line_ML_y', 'Best_Line_ML_y',
       'Worst_Line_ML_y', 'ABS_Line_ML'],
      dtype='object')

In [ ]:
data = single_game
# [['PTS_QTR1_x',
#        'PTS_QTR2_x', 'PTS_QTR3_x', 'PTS_QTR4_x', 'PTS_x', 'FG_PCT_x',
#        'FT_PCT_x', 'FG3_PCT_x', 'AST_x', 'REB_x', 'TOV_x', 'PTS_QTR1_y',
#        'PTS_QTR2_y', 'PTS_QTR3_y', 'PTS_QTR4_y', 'PTS_y', 'FG_PCT_y',
#        'FT_PCT_y', 'FG3_PCT_y', 'AST_y', 'REB_y', 'TOV_y','ABS_Line_ML']]

data['Points_Q1'] = abs(data['PTS_QTR1_x'] - data['PTS_QTR1_y'])
data['Points_Q2'] = abs(data['PTS_QTR2_x'] - data['PTS_QTR2_y'])
data['Points_Q3'] = abs(data['PTS_QTR3_x'] - data['PTS_QTR3_y'])
data['Points_Q4'] = abs(data['PTS_QTR4_x'] - data['PTS_QTR4_y'])

data['PTS'] = abs(data['PTS_x'] - data['PTS_y'])
data['FG_PCT'] = abs(data['FG_PCT_x'] - data['FG_PCT_y'])
data['FT_PCT'] = abs(data['FT_PCT_x'] - data['FT_PCT_y'])
data['FT_PCT3'] = abs(data['FG3_PCT_x'] - data['FG3_PCT_y'])
data['AST'] = abs(data['AST_x'] - data['AST_y'])
data['REB'] = abs(data['REB_x'] - data['REB_y'])
data['TO'] = abs(data['TOV_x'] - data['TOV_y'])


data_avg = data[['Points_Q1','Points_Q2','Points_Q3','Points_Q4','PTS','FG_PCT','FT_PCT','FT_PCT3','AST','REB','TO','Open_Line_ML_x','ABS_Line_ML']]

data_avg

,Points_Q1,Points_Q2,Points_Q3,Points_Q4,PTS,FG_PCT,FT_PCT,FT_PCT3,AST,REB,TO,Open_Line_ML_x,ABS_Line_ML
0,7.000000,4.000000,1.000000,2.000000,10.000000,0.100000,0.082000,0.100000,4.000000,15.000000,8.000000,220.0,455.0
1,6.000000,2.000000,9.000000,4.000000,13.000000,0.024000,0.008000,0.038000,1.000000,5.000000,7.000000,220.0,504.8
2,4.000000,6.000000,6.000000,0.000000,8.000000,0.023000,0.391000,0.102000,2.000000,6.000000,3.000000,350.0,886.6
3,2.000000,2.000000,2.000000,0.000000,6.000000,0.013000,0.055000,0.131000,7.000000,6.000000,1.000000,230.0,503.2
4,3.000000,4.000000,7.000000,4.000000,4.000000,0.037000,0.259000,0.065000,4.000000,0.000000,2.000000,-280.0,560.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,2.390244,2.414634,0.682927,0.658537,7.060976,0.013780,0.012122,0.007756,2.256098,0.280488,0.195122,285.0,707.8
864,0.829268,1.109756,2.439024,2.207317,7.280488,0.041622,0.045976,0.076598,2.707317,0.719512,0.000000,380.0,914.6
865,2.512195,1.146341,1.829268,1.207317,6.841463,0.045756,0.093939,0.020817,1.500000,0.317073,1.024390,165.0,294.2
866,0.560976,1.695122,0.121951,0.939024,0.853659,0.029951,0.048256,0.003671,3.048780,1.036585,0.243902,-260.0,530.4


In [ ]:
data_avg

,Points_Q1,Points_Q2,Points_Q3,Points_Q4,PTS,FG_PCT,FT_PCT,FT_PCT3,AST,REB,TO,Open_Line_ML_x,ABS_Line_ML
0,7.000000,4.000000,1.000000,2.000000,10.000000,0.100000,0.082000,0.100000,4.000000,15.000000,8.000000,220.0,455.0
1,6.000000,2.000000,9.000000,4.000000,13.000000,0.024000,0.008000,0.038000,1.000000,5.000000,7.000000,220.0,504.8
2,4.000000,6.000000,6.000000,0.000000,8.000000,0.023000,0.391000,0.102000,2.000000,6.000000,3.000000,350.0,886.6
3,2.000000,2.000000,2.000000,0.000000,6.000000,0.013000,0.055000,0.131000,7.000000,6.000000,1.000000,230.0,503.2
4,3.000000,4.000000,7.000000,4.000000,4.000000,0.037000,0.259000,0.065000,4.000000,0.000000,2.000000,-280.0,560.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,2.390244,2.414634,0.682927,0.658537,7.060976,0.013780,0.012122,0.007756,2.256098,0.280488,0.195122,285.0,707.8
864,0.829268,1.109756,2.439024,2.207317,7.280488,0.041622,0.045976,0.076598,2.707317,0.719512,0.000000,380.0,914.6
865,2.512195,1.146341,1.829268,1.207317,6.841463,0.045756,0.093939,0.020817,1.500000,0.317073,1.024390,165.0,294.2
866,0.560976,1.695122,0.121951,0.939024,0.853659,0.029951,0.048256,0.003671,3.048780,1.036585,0.243902,-260.0,530.4


### Sanity Check Intuition

In [ ]:
# Look Seperate the data into three categories based on the combined spread


a = data_avg[data_avg['ABS_Line_ML'] <= 400 ] 

b = data_avg[data_avg['ABS_Line_ML'] > 400 ] 
b = b[b['ABS_Line_ML'] <= 650 ]


c = data_avg[data_avg['ABS_Line_ML'] > 650 ]

print("###################################")
print("Average Odds Spread less than 400")
print("###################################")
print(a.mean())
print("###################################")
print("Average Odds Spread between 400 and 650")
print("###################################")
print(b.mean())
print("###################################")
print("Average Odds Spread greater than 65")
print("###################################")
print(c.mean())

###################################
Average Odds Spread less than 400
###################################
Points_Q1           1.633291
Points_Q2           1.577200
Points_Q3           1.380777
Points_Q4           1.453898
PTS                 4.354428
FG_PCT              0.020056
FT_PCT              0.040382
FT_PCT3             0.028033
AST                 1.644884
REB                 2.411660
TO                  1.169235
Open_Line_ML_x      5.066092
ABS_Line_ML       322.835632
dtype: float64
###################################
Average Odds Spread between 400 and 650
###################################
Points_Q1           1.759229
Points_Q2           1.865452
Points_Q3           1.575270
Points_Q4           1.605544
PTS                 4.631077
FG_PCT              0.022800
FT_PCT              0.042585
FT_PCT3             0.030175
AST                 1.751235
REB                 2.450113
TO                  1.377309
Open_Line_ML_x    100.627010
ABS_Line_ML       505.722830
dtype: float6

### Auto ML, PyCaret

In [ ]:
from pycaret.regression import *

reg = setup(data_avg, target='ABS_Line_ML', session_id=123, log_experiment=False, experiment_name='FullData')

best = compare_models()

,Description,Value
0,Session id,123
1,Target,ABS_Line_ML
2,Target type,Regression
3,Data shape,"(868, 13)"
4,Train data shape,"(607, 13)"
5,Test data shape,"(261, 13)"
6,Numeric features,12
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,59.2902,7750.5858,86.6283,0.8452,0.1645,0.1239,0.3910
et,Extra Trees Regressor,59.5237,7916.3440,87.6612,0.8420,0.1645,0.1246,0.2020
gbr,Gradient Boosting Regressor,59.9801,8130.7072,88.7232,0.8373,0.1663,0.1246,0.1690
knn,K Neighbors Regressor,58.8625,8261.9896,89.0595,0.8370,0.1661,0.1217,0.0290
lightgbm,Light Gradient Boosting Machine,64.9790,9090.7860,94.3298,0.8161,0.1776,0.1353,0.1370
ada,AdaBoost Regressor,84.5334,11903.1460,108.3488,0.7552,0.2292,0.1996,0.1140
dt,Decision Tree Regressor,82.3229,13725.8597,114.8627,0.7261,0.2230,0.1673,0.0300
lr,Linear Regression,157.4665,41885.3152,203.8224,0.1249,0.3969,0.3502,0.4670
lar,Least Angle Regression,157.4664,41885.2959,203.8224,0.1249,0.3969,0.3502,0.0280
llar,Lasso Least Angle Regression,161.5473,42920.4646,206.2816,0.1131,0.3921,0.3579,0.0270


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
mlp = MLPRegressor(hidden_layer_sizes=(100,100), \
                   activation='relu',
                   solver='adam',
                   alpha=0.0001,
                   batch_size='auto',
                   learning_rate='constant',
                   learning_rate_init=0.001,
                   power_t=0.5,
                   max_iter=200,
                   shuffle=True,
                   random_state=None,
                   tol=0.0001,
                   verbose=False,
                   warm_start=False,
                   momentum=0.9,
                   nesterovs_momentum=True,
                   early_stopping=False,
                   validation_fraction=0.1,
                   beta_1=0.9,
                   beta_2=0.999,
                   epsilon=1e-08,
                   n_iter_no_change=10,
                   max_fun=15000)

mlp

MLPRegressor(hidden_layer_sizes=(100, 100))

In [ ]:
data_avg.columns

Index(['Points_Q1', 'Points_Q2', 'Points_Q3', 'Points_Q4', 'PTS', 'FG_PCT',
       'FT_PCT', 'FT_PCT3', 'AST', 'REB', 'TO', 'Open_Line_ML_x',
       'ABS_Line_ML'],
      dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#dat_avga_X = data[['TeamId_x', 'TeamId_y']]
data_x = data_avg[['Points_Q1', 'Points_Q2', 'Points_Q3', 'Points_Q4', 'PTS', 'FG_PCT', 'FT_PCT', 'FT_PCT3', 'AST', 'REB', 'TO', 'Open_Line_ML_x','ABS_Line_ML']]
data_y = data_avg[['ABS_Line_ML']]

teamid_train, teamid_test, odds_train, odds_test = train_test_split(data_x, data_y, test_size = .1)


In [ ]:
mlp.fit(teamid_train, odds_train)

MLPRegressor(hidden_layer_sizes=(100, 100))

In [ ]:
mlp.score(teamid_train, odds_train)

0.9999975486759454

In [ ]:
mlp.score(teamid_test, odds_test)

0.9999977648896456

In [ ]:
datasets = glob("*.csv")
datasets

['2012-2013_RawScores_VegasOdds.csv',
 '2012_2013_Cumulative_Data.csv',
 '2012_2013_duplicate_rows.csv',
 '2012_2013_add_noise.csv',
 '2012_2013_bootstrapped.csv',
 '2012_2013_drop_features.csv',
 '2012_2013_add_dummy_cols.csv']

In [ ]:
# Loop through datasets
# del data

for p in datasets[1:]:

  print(p)
  data = pd.read_csv(p)
  data = data.dropna()
  single_game = merge_data(data,'GameId')
  data = single_game[['PTS_QTR1_x',
        'PTS_QTR2_x', 'PTS_QTR3_x', 'PTS_QTR4_x', 'PTS_x', 'FG_PCT_x',
        'FT_PCT_x', 'FG3_PCT_x', 'AST_x', 'REB_x', 'TOV_x', 'PTS_QTR1_y',
        'PTS_QTR2_y', 'PTS_QTR3_y', 'PTS_QTR4_y', 'PTS_y', 'FG_PCT_y',
        'FT_PCT_y', 'FG3_PCT_y', 'AST_y', 'REB_y', 'TOV_y','Open_Line_ML_x','ABS_Line_ML']]

  data['Points_Q1'] = abs(data['PTS_QTR1_x'] - data['PTS_QTR1_y'])
  data['Points_Q2'] = abs(data['PTS_QTR2_x'] - data['PTS_QTR2_y'])
  data['Points_Q3'] = abs(data['PTS_QTR3_x'] - data['PTS_QTR3_y'])
  data['Points_Q4'] = abs(data['PTS_QTR4_x'] - data['PTS_QTR4_y'])

  data['PTS'] = abs(data['PTS_x'] - data['PTS_y'])
  data['FG_PCT'] = abs(data['FG_PCT_x'] - data['FG_PCT_y'])
  data['FT_PCT'] = abs(data['FT_PCT_x'] - data['FT_PCT_y'])
  data['FT_PCT3'] = abs(data['FG3_PCT_x'] - data['FG3_PCT_y'])
  data['AST'] = abs(data['AST_x'] - data['AST_y'])
  data['REB'] = abs(data['REB_x'] - data['REB_y'])
  data['TO'] = abs(data['TOV_x'] - data['TOV_y'])


  data_avg = data[['Points_Q1','Points_Q2','Points_Q3','Points_Q4','PTS','FG_PCT','FT_PCT','FT_PCT3','AST','REB','TO','Open_Line_ML_x','ABS_Line_ML']]

  reg = setup(data_avg, target='ABS_Line_ML', session_id=123, log_experiment=False, experiment_name='FullData')
  best = compare_models()


2012_2013_Cumulative_Data.csv


,Description,Value
0,Session id,123
1,Target,ABS_Line_ML
2,Target type,Regression
3,Data shape,"(868, 13)"
4,Train data shape,"(607, 13)"
5,Test data shape,"(261, 13)"
6,Numeric features,12
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,59.2902,7750.5858,86.6283,0.8452,0.1645,0.1239,0.3900
et,Extra Trees Regressor,59.5237,7916.3440,87.6612,0.8420,0.1645,0.1246,0.2130
gbr,Gradient Boosting Regressor,59.9801,8130.7072,88.7232,0.8373,0.1663,0.1246,0.1710
knn,K Neighbors Regressor,58.8625,8261.9896,89.0595,0.8370,0.1661,0.1217,0.0290
lightgbm,Light Gradient Boosting Machine,64.9790,9090.7860,94.3298,0.8161,0.1776,0.1353,0.1410
ada,AdaBoost Regressor,84.5334,11903.1460,108.3488,0.7552,0.2292,0.1996,0.1170
dt,Decision Tree Regressor,82.3229,13725.8597,114.8627,0.7261,0.2230,0.1673,0.0290
lr,Linear Regression,157.4665,41885.3152,203.8224,0.1249,0.3969,0.3502,0.2920
lar,Least Angle Regression,157.4664,41885.2959,203.8224,0.1249,0.3969,0.3502,0.0280
llar,Lasso Least Angle Regression,161.5473,42920.4646,206.2816,0.1131,0.3921,0.3579,0.0270


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

2012_2013_duplicate_rows.csv


...

,Description,Value
0,Session id,123
1,Target,ABS_Line_ML
2,Target type,Regression
3,Data shape,"(1229, 13)"
4,Train data shape,"(860, 13)"
5,Test data shape,"(369, 13)"
6,Numeric features,12
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
llar,Lasso Least Angle Regression,25.3915,1016.5673,31.8234,-0.0134,0.5309,0.5788,0.0250
dummy,Dummy Regressor,25.3915,1016.5673,31.8234,-0.0134,0.5309,0.5788,0.0250
br,Bayesian Ridge,25.4932,1023.7674,31.9352,-0.0206,0.5324,0.5807,0.0280
en,Elastic Net,25.5014,1026.4016,31.9776,-0.0236,0.5329,0.5812,0.0260
lasso,Lasso Regression,25.5549,1029.2775,32.0230,-0.0265,0.5333,0.5821,0.0300
omp,Orthogonal Matching Pursuit,25.6167,1032.9919,32.0836,-0.0303,0.5337,0.5826,0.0270
ada,AdaBoost Regressor,25.8105,1035.0946,32.1168,-0.0341,0.5369,0.5935,0.0950
ridge,Ridge Regression,25.5896,1036.4064,32.1356,-0.0342,0.5348,0.5836,0.0280
lar,Least Angle Regression,25.5920,1036.6123,32.1388,-0.0344,0.5349,0.5837,0.0290
lr,Linear Regression,25.5920,1036.6123,32.1388,-0.0344,0.5349,0.5837,0.0270


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

2012_2013_drop_features.csv


KeyError: ignored

In [ ]:

data = pd.read_csv('2012_2013_drop_features.csv')
data = data.dropna()
single_game = merge_data(data,'GameId')

single_game.columns

data = single_game[['PTS_QTR1_x',
      'PTS_QTR2_x', 'PTS_QTR3_x', 'PTS_x', 'FG_PCT_x',
      'FT_PCT_x', 'FG3_PCT_x', 'AST_x', 'REB_x', 'TOV_x', 'PTS_QTR1_y',
      'PTS_QTR2_y', 'PTS_QTR3_y', 'PTS_y', 'FG_PCT_y',
      'FT_PCT_y', 'FG3_PCT_y', 'AST_y', 'REB_y', 'TOV_y','Open_Line_ML_x','ABS_Line_ML']]
  

data['Points_Q1'] = abs(data['PTS_QTR1_x'] - data['PTS_QTR1_y'])
data['Points_Q2'] = abs(data['PTS_QTR2_x'] - data['PTS_QTR2_y'])
data['Points_Q3'] = abs(data['PTS_QTR3_x'] - data['PTS_QTR3_y'])
#data['Points_Q4'] = abs(data['PTS_QTR4_x'] - data['PTS_QTR4_y'])

data['PTS'] = abs(data['PTS_x'] - data['PTS_y'])
data['FG_PCT'] = abs(data['FG_PCT_x'] - data['FG_PCT_y'])
data['FT_PCT'] = abs(data['FT_PCT_x'] - data['FT_PCT_y'])
data['FT_PCT3'] = abs(data['FG3_PCT_x'] - data['FG3_PCT_y'])
data['AST'] = abs(data['AST_x'] - data['AST_y'])
data['REB'] = abs(data['REB_x'] - data['REB_y'])
data['TO'] = abs(data['TOV_x'] - data['TOV_y'])


data_avg = data[['Points_Q1','Points_Q2','Points_Q3','PTS','FG_PCT','FT_PCT','FT_PCT3','AST','REB','TO','Open_Line_ML_x','ABS_Line_ML']]

reg = setup(data_avg, target='ABS_Line_ML', session_id=123, log_experiment=False, experiment_name='FullData')
best = compare_models()




,Description,Value
0,Session id,123
1,Target,ABS_Line_ML
2,Target type,Regression
3,Data shape,"(868, 12)"
4,Train data shape,"(607, 12)"
5,Test data shape,"(261, 12)"
6,Numeric features,11
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
omp,Orthogonal Matching Pursuit,422.1936,272547.0180,519.7997,-0.0094,0.6995,0.9082,0.0250
llar,Lasso Least Angle Regression,422.3477,273705.6406,520.8769,-0.0132,0.7005,0.9114,0.0260
br,Bayesian Ridge,422.2400,274717.2680,521.8739,-0.0172,0.7018,0.9163,0.0260
dummy,Dummy Regressor,422.8141,274696.4578,521.8715,-0.0173,0.7018,0.9166,0.0230
en,Elastic Net,424.1043,275715.3484,522.7749,-0.0209,0.7023,0.9127,0.0250
lasso,Lasso Regression,424.9456,276630.9125,523.6420,-0.0244,0.7032,0.9141,0.0300
ridge,Ridge Regression,426.0451,277933.2094,524.9419,-0.0298,0.7042,0.9154,0.0260
lr,Linear Regression,427.4475,279947.8938,526.9394,-0.0383,0.7059,0.9177,0.4120
lar,Least Angle Regression,427.4475,279947.8875,526.9394,-0.0383,0.7059,0.9177,0.0270
rf,Random Forest Regressor,429.8908,281511.7192,528.2133,-0.0442,0.7079,0.9423,0.4190


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

In [ ]:

data = pd.read_csv('2012_2013_add_dummy_cols.csv')
#print(data.columns)
data = data.dropna()
single_game = merge_data(data,'GameId')
#print(single_game.columns)

data = single_game[['PTS_QTR1_x',
      'PTS_QTR2_x', 'PTS_QTR3_x', 'PTS_QTR4_x', 'PTS_x', 'FG_PCT_x',
      'FT_PCT_x', 'FG3_PCT_x', 'AST_x', 'REB_x', 'TOV_x', 'PTS_QTR1_y',
      'PTS_QTR2_y', 'PTS_QTR3_y', 'PTS_QTR4_y', 'PTS_y', 'FG_PCT_y',
      'FT_PCT_y', 'FG3_PCT_y', 'AST_y', 'REB_y', 'TOV_y','Open_Line_ML_x','ABS_Line_ML','10000_x','10001_x']]

data['Points_Q1'] = abs(data['PTS_QTR1_x'] - data['PTS_QTR1_y'])
data['Points_Q2'] = abs(data['PTS_QTR2_x'] - data['PTS_QTR2_y'])
data['Points_Q3'] = abs(data['PTS_QTR3_x'] - data['PTS_QTR3_y'])
data['Points_Q4'] = abs(data['PTS_QTR4_x'] - data['PTS_QTR4_y'])

data['PTS'] = abs(data['PTS_x'] - data['PTS_y'])
data['FG_PCT'] = abs(data['FG_PCT_x'] - data['FG_PCT_y'])
data['FT_PCT'] = abs(data['FT_PCT_x'] - data['FT_PCT_y'])
data['FT_PCT3'] = abs(data['FG3_PCT_x'] - data['FG3_PCT_y'])
data['AST'] = abs(data['AST_x'] - data['AST_y'])
data['REB'] = abs(data['REB_x'] - data['REB_y'])
data['TO'] = abs(data['TOV_x'] - data['TOV_y'])


data.columns

data_avg = data[['Points_Q1','Points_Q2','Points_Q3','Points_Q4','PTS','FG_PCT','FT_PCT','FT_PCT3','AST','REB','TO','Open_Line_ML_x','ABS_Line_ML','10000_x','10001_x']]

reg = setup(data_avg, target='ABS_Line_ML', session_id=123, log_experiment=False, experiment_name='FullData')
best = compare_models()


,Description,Value
0,Session id,123
1,Target,ABS_Line_ML
2,Target type,Regression
3,Data shape,"(868, 15)"
4,Train data shape,"(607, 15)"
5,Test data shape,"(261, 15)"
6,Numeric features,14
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
omp,Orthogonal Matching Pursuit,422.1936,272547.0180,519.7997,-0.0094,0.6995,0.9082,0.0270
llar,Lasso Least Angle Regression,423.0115,274604.2438,521.6965,-0.0163,0.7014,0.9132,0.0290
br,Bayesian Ridge,422.2396,274716.9301,521.8735,-0.0172,0.7018,0.9163,0.0290
dummy,Dummy Regressor,422.8141,274696.4578,521.8715,-0.0173,0.7018,0.9166,0.0240
en,Elastic Net,425.5028,277213.8516,524.1569,-0.0264,0.7040,0.9167,0.0270
lasso,Lasso Regression,426.8751,278675.3641,525.5482,-0.0320,0.7055,0.9196,0.0310
ridge,Ridge Regression,427.9487,279962.3000,526.8329,-0.0374,0.7065,0.9208,0.0270
lr,Linear Regression,429.0356,281884.0031,528.7434,-0.0456,0.7080,0.9227,0.7700
lar,Least Angle Regression,429.0356,281883.9875,528.7433,-0.0456,0.7080,0.9227,0.0270
rf,Random Forest Regressor,431.3842,283458.1538,530.3650,-0.0544,0.7108,0.9471,0.4960


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

In [ ]:
data.columns

Index(['PTS_QTR1_x', 'PTS_QTR2_x', 'PTS_QTR3_x', 'PTS_QTR4_x', 'PTS_x',
       'FG_PCT_x', 'FT_PCT_x', 'FG3_PCT_x', 'AST_x', 'REB_x', 'TOV_x',
       'PTS_QTR1_y', 'PTS_QTR2_y', 'PTS_QTR3_y', 'PTS_QTR4_y', 'PTS_y',
       'FG_PCT_y', 'FT_PCT_y', 'FG3_PCT_y', 'AST_y', 'REB_y', 'TOV_y',
       'Open_Line_ML_x', 'ABS_Line_ML', 'Points_Q1', 'Points_Q2', 'Points_Q3',
       'Points_Q4', 'PTS', 'FG_PCT', 'FT_PCT', 'FT_PCT3', 'AST', 'REB', 'TO'],
      dtype='object')